### Creating footfall data per day per medicine for train dataset

- The strategy is to find the footfall per day per medicine
- So how do we find it ?
- we do  groupby w.r.t date and take sum of sales
- we get a dataframe containing total sales per day
- we merge it with actual dataframe
- we create a column by diving actual sales with total sales per day
- we get sales contribution of each day sales
- we multiply sales contibution with footfall and we get footfall per day per medicine

In [11]:
import numpy as np
import pandas as pd
import warnings
import os
warnings.filterwarnings('ignore')

In [32]:
path=os.getcwd()

In [13]:
train=pd.read_csv("train_finaldata.csv", sep=",", na_values=["?",",","#","NaN","unknown",""])
train["discounted"]=train["discounted"].fillna(0)

In [14]:
train.discounted=train.discounted.astype("int")

In [15]:
train.head()

,year,month,day,city,medicine,date,sales,discounted,footfall,year_month,city_medicine,week
0,2015,1,2,1,1,2015-01-02,24.0,0,13948.0,2015_01,1_1,1
1,2015,1,2,1,2,2015-01-02,144.0,0,13948.0,2015_01,1_2,1
2,2015,1,2,1,3,2015-01-02,84.0,0,13948.0,2015_01,1_3,1
3,2015,1,2,1,4,2015-01-02,24.0,0,13948.0,2015_01,1_4,1
4,2015,1,2,1,5,2015-01-02,8.0,0,13948.0,2015_01,1_5,1


In [16]:
trn_grp_date=train.groupby(["date"])["sales"].sum().reset_index()# groupby date and take sum of sales

In [17]:
trn_grp_date.rename(columns={"sales":"total_sales_per_day"},inplace=True)# rename sales to total_sales_per_day

In [18]:
trn_grp_date.head(2)

,date,total_sales_per_day
0,2015-01-02,867673.604
1,2015-01-03,586097.240


In [19]:
trn_grp_date.shape

(1269, 2)

In [20]:
train=train.merge(trn_grp_date,on=["date"],how="left")

In [21]:
train.shape

(22601741, 13)

In [22]:
train.head(2)

,year,month,day,city,medicine,date,sales,discounted,footfall,year_month,city_medicine,week,total_sales_per_day
0,2015,1,2,1,1,2015-01-02,24.0,0,13948.0,2015_01,1_1,1,867673.604
1,2015,1,2,1,2,2015-01-02,144.0,0,13948.0,2015_01,1_2,1,867673.604


In [23]:
train["sales_contribution"]=train["sales"]/train["total_sales_per_day"]

In [24]:
train.head(2)

,year,month,day,city,medicine,date,sales,discounted,footfall,year_month,city_medicine,week,total_sales_per_day,sales_contribution
0,2015,1,2,1,1,2015-01-02,24.0,0,13948.0,2015_01,1_1,1,867673.604,0.000028
1,2015,1,2,1,2,2015-01-02,144.0,0,13948.0,2015_01,1_2,1,867673.604,0.000166


In [25]:
train["footfall_perday_permed"]=train["sales_contribution"]*train["footfall"]# creating footfall per day per medicine

In [26]:
for col in ["year","month","day","city","medicine","week"]:
    train[col]=train[col].astype(int)

In [27]:
train.dtypes

year                        int32
month                       int32
day                         int32
city                        int32
medicine                    int32
date                       object
sales                     float64
discounted                  int32
footfall                  float64
year_month                 object
city_medicine              object
week                        int32
total_sales_per_day       float64
sales_contribution        float64
footfall_perday_permed    float64
dtype: object

In [28]:
train.drop(["total_sales_per_day","sales_contribution","footfall"],inplace=True,axis=1)

In [30]:
train.head()

,year,month,day,city,medicine,date,sales,discounted,year_month,city_medicine,week,footfall_perday_permed
0,2015,1,2,1,1,2015-01-02,24.0,0,2015_01,1_1,1,0.385804
1,2015,1,2,1,2,2015-01-02,144.0,0,2015_01,1_2,1,2.314824
2,2015,1,2,1,3,2015-01-02,84.0,0,2015_01,1_3,1,1.350314
3,2015,1,2,1,4,2015-01-02,24.0,0,2015_01,1_4,1,0.385804
4,2015,1,2,1,5,2015-01-02,8.0,0,2015_01,1_5,1,0.128601


In [31]:
train.to_csv("train_footfalldata.csv",index=False)# saving train along with footfall data